In [3]:
! pip install glaredb --upgrade

In [4]:
import glaredb

In [5]:
con = glaredb.connect()

In [6]:
con.sql(
    f"""   
        SELECT * FROM read_csv(
            's3://glaredb-sandbox/sales_data.csv',
            access_key_id => '{aws_access_key_id}',
            secret_access_key => '{aws_secret_access_key}',
            region =>'us-east-2'
        ) LIMIT 10;
    """
).to_pandas()

,Date,Store_ID,Product,Quantity,Unit_Price,Total_Sale
0,2023-01-01,9,Desk Calendar,14,34.88,488.30
1,2023-01-01,12,File Folders,31,23.72,735.23
2,2023-01-02,13,Desk Calendar,42,17.52,735.93
3,2023-01-02,11,Ballpoint Pens,39,86.27,3364.50
4,2023-01-03,6,Desk Lamp,43,25.38,1091.45
5,2023-01-03,3,Whiteboard,14,96.05,1344.68
6,2023-01-03,10,USB Drive,10,46.29,462.86
7,2023-01-03,5,Ballpoint Pens,27,54.06,1459.68
8,2023-01-03,3,File Folders,14,55.58,778.06
9,2023-01-03,15,Office Chair,1,28.01,28.01


In [7]:
# gcs_service_account = r"""
# {
#   "type": "service_account",
#   "project_id": "glaredb-demo",
#   "private_key_id": "private_key_id",
#   "private_key": "private_key",
#   "client_email": "hello@gmail.com",
#   "client_id": "client_id",
#   "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#   "token_uri": "https://oauth2.googleapis.com/token",
#   "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
#   "client_x509_cert_url": "cert",
#   "universe_domain": "googleapis.com"
# }
# """

con.sql(
    f"""
        SELECT * FROM read_parquet(
            'gs://glaredb_dbt_public_demo/store_info.parquet',
            service_account_key => '{gcs_service_account}'
        ) LIMIT 10
    """
).to_pandas()

,Store_ID,Location,Size_sqft,Opening_Date,Manager_Name
0,1,"Chicago, IL",17673,2010-01-01,Manager_1
1,2,"Columbus, OH",5073,2010-05-01,Manager_2
2,3,"Houston, TX",13585,2010-08-29,Manager_3
3,4,"San Diego, CA",18079,2010-12-27,Manager_4
4,5,"Fort Worth, TX",9330,2011-04-26,Manager_5
5,6,"Dallas, TX",15032,2011-08-24,Manager_6
6,7,"San Francisco, CA",18321,2011-12-22,Manager_7
7,8,"Phoenix, AZ",13798,2012-04-20,Manager_8
8,9,"Philadelphia, PA",15370,2012-08-18,Manager_9
9,10,"Austin, TX",14721,2012-12-16,Manager_10


In [8]:
con.sql(
    f"""
        SELECT 
            sum(sales."Total_Sale") all_sales,
            stores."Location",
            stores."Manager_Name"
        FROM 
            read_csv(
                's3://glaredb-sandbox/sales_data.csv',
                access_key_id => '{aws_access_key_id}',
                secret_access_key => '{aws_secret_access_key}',
                region =>'us-east-2'
            ) sales
        JOIN
            read_parquet(
                'gs://glaredb_dbt_public_demo/store_info.parquet',
                service_account_key => '{gcs_service_account}'
            ) stores
        ON 
            sales."Store_ID" = stores."Store_ID"
        GROUP BY
            stores."Location",
            stores."Manager_Name"
        ORDER BY all_sales DESC
    """
).to_pandas()

,all_sales,Location,Manager_Name
0,113289.21,"Columbus, OH",Manager_2
1,108244.57,"Houston, TX",Manager_3
2,100384.51,"New York, NY",Manager_12
3,97008.80,"Chicago, IL",Manager_1
4,96528.63,"Jacksonville, FL",Manager_15
5,96517.49,"Fort Worth, TX",Manager_5
6,96143.64,"San Jose, CA",Manager_13
7,93136.80,"Philadelphia, PA",Manager_9
8,89745.83,"Phoenix, AZ",Manager_8
9,83562.25,"Dallas, TX",Manager_6
